In [1]:
import numpy as np
import pandas as pd
import re

In [2]:
wm_crime = pd.read_csv('wm_crime.csv')
wmp_lsoa = pd.read_csv('wmp_areas.csv')
avg_temp = pd.read_csv('avg_temp.csv')
med_price = pd.read_csv('med_house.csv')
uni_credit = pd.read_csv('uni_credit.csv')

In [3]:
# NaN value to string
wm_crime["LSOA name"] = wm_crime["LSOA name"].fillna('NaN')

In [4]:
# delete LSOAs that are not covered by WMP and NaNs
district_list = ["Birmingham", "Coventry", "Dudley", "Sandwell", "Solihull", "Walsall", "Wolverhampton"]
data_lsoa = wm_crime["LSOA name"].unique()
lsoa_not_in = []

for lsoa in data_lsoa:
    if re.findall(r"(?=("+'|'.join(district_list)+r"))", lsoa):
        pass
    else:
        lsoa_not_in.append(lsoa)
                
filt = wm_crime[wm_crime['LSOA name'].isin(lsoa_not_in)].index
wm_crime.drop(filt, inplace = True)

"This is the list of LSOA : {} not covered by WMP and removed.".format(lsoa_not_in)

'This is the list of LSOA : [\'Bournemouth 020A\', \'Bromsgrove 003D\', \'Bromsgrove 004C\', \'Bromsgrove 005A\', \'Ceredigion 009B\', \'Cheshire West and Chester 039D\', \'County Durham 022E\', \'East Dorset 005D\', \'East Dorset 012E\', \'Fylde 004E\', \'Harborough 007E\', \'Isle of Wight 013A\', \'Isle of Wight 018C\', \'Lichfield 006E\', \'Lichfield 012A\', \'North Warwickshire 004A\', \'North Warwickshire 006D\', \'North Warwickshire 007D\', \'Nuneaton and Bedworth 015E\', \'Nuneaton and Bedworth 018A\', \'Nuneaton and Bedworth 018D\', \'Nuneaton and Bedworth 018E\', \'Powys 006A\', \'Powys 010A\', \'Purbeck 001B\', \'Purbeck 006D\', \'Richmondshire 001B\', \'Rother 002C\', \'Shropshire 029E\', \'Shropshire 037D\', \'South Hams 011A\', \'South Staffordshire 009E\', \'South Staffordshire 012B\', \'Swindon 026F\', \'Test Valley 009C\', \'Warwick 004A\', \'Warwick 005G\', \'West Oxfordshire 002A\', \'Worcester 008B\', \'Wychavon 007A\', \'NaN\', \'Barnet 020B\', \'Barnet 025C\', \'Ba

In [5]:
# check if LSOA in crime data now matches LSOA in wmp_lsoa
print(len(wmp_lsoa.name))
print(len(wm_crime["LSOA name"].unique()))

1680
1680


### The dataset

In [6]:
# change to datetime
wm_crime["Month"] = pd.to_datetime(wm_crime["Month"])

# drop columns
wm_crime.drop(columns = ['Crime ID', 'Reported by', 'Falls within', 'Longitude', 'Latitude', 'Location', 
                         'LSOA code', 'Last outcome category', 'Context'], axis = 1, inplace =True)
# rename columns
wm_crime.rename(columns = {'LSOA name': 'LSOA', 'Crime type':'Type'}, inplace = True)

In [7]:
from itertools import product

# all the unique combination of features
month_list = wm_crime.Month.unique()
LSOA_list = wm_crime.LSOA.unique()
type_list = wm_crime.Type.unique()

combination_list = list(product(month_list, LSOA_list, type_list))
comb_df = pd.DataFrame(combination_list, columns = wm_crime.columns)

crime1 = wm_crime.groupby(['Month','LSOA', 'Type']).size().reset_index(name = 'Total')

# merge two df to fill values
crime1 = pd.merge(crime1, comb_df, how = 'right', on = ['Month', 'LSOA', 'Type'])

#fill NaNs with 0 to indicate that there were no crime reported
crime1["Total"] = crime1["Total"].fillna(0)

In [8]:
# add avg_temp data to crime data
avg_temp.rename(columns = {'Unnamed: 0': 'Month'}, inplace = True)
avg_temp.Month = pd.to_datetime(avg_temp.Month)

mapping = dict(avg_temp[['Month', 'Avg_temp']].values)
crime1['Avg_temp'] = crime1.Month.map(mapping)

crime1.Avg_temp =[int(i.replace('°C', '')) for i in crime1.Avg_temp]

In [9]:
# adding median property price data
med_price = med_price.sort_values(by = "LSOA name")
lsoa_list = med_price["LSOA name"]
comb_list = list(product(month_list, lsoa_list))
price_df = pd.DataFrame(comb_list, columns = ['Month', 'LSOA'])
price_df = price_df.sort_values(by = ['Month', 'LSOA'])

col_n = 4
price_list = []

while col_n < 14:
    col_n += 1
    if col_n == 5:
        for i in range(1, 3):
            for j in list(med_price.iloc[:, col_n]):
                price_list.append(j)
    elif (col_n > 5) & (col_n < 14):
        for i in range(1, 4):
            for j in list(med_price.iloc[:, col_n]):
                price_list.append(j)
    else:
        for i in range(1, 11):
            for j in list(med_price.iloc[:, col_n]):
                price_list.append(j)

price_df['Med_price'] = price_list 

# merge two df to add data points and fill label values
crime1.sort_values(by = ['Month', 'LSOA'])
crime1 = pd.merge(crime1, price_df, how = 'right', on = ['Month', 'LSOA'])

# missing value imputation with average of median price over any period available for each LSOA 
# replace missing value indicator ":" with np.nan
crime1['Med_price'] = crime1['Med_price'].replace(":", np.nan)
crime1['Med_price'] = [float(i) for i in crime1['Med_price']]
crime1.Med_price = crime1.groupby('LSOA')['Med_price'].apply(lambda x: x.fillna(x.mean()))

In [25]:
missing_price = ['Birmingham 050E', 'Birmingham 053B', 'Birmingham 071A','Birmingham 077A', 'Birmingham 109A',
                 'Birmingham 136A','Coventry 007F', 'Coventry 024C', 'Wolverhampton 007B','Wolverhampton 015C'] 

# imputing median price for areas without any historical records
# by averaging over areas within corresponding MLSOA 
MLSOA = [i[:-1] for i in missing_price]
LSOA_list = crime1.LSOA.unique()
impute_dict = {mlsoa:[] for mlsoa in MLSOA}
for lsoa in LSOA_list:
    for mlsoa in MLSOA:
        if mlsoa in lsoa:
            impute_dict[mlsoa] = impute_dict.get(mlsoa, []) + [lsoa]            

m_value = pd.DataFrame()

for mlsoa in MLSOA:
    impute_df = crime1[crime1['LSOA'].isin(impute_dict[mlsoa])]
    m = impute_df.groupby('Month')['Med_price'].mean()
    m_value[mlsoa] = m

m_value.columns = missing_price

for lsoa in m_value.columns:
    crime1.loc[(crime1.Med_price.isnull()) & (crime1.LSOA == lsoa ), 'Med_price' ] = crime1['Month'].map(m_value[lsoa])

In [41]:
from itertools import repeat
# adding universal credit data
uni_credit = uni_credit.sort_values(by = 'LSOA') 
uc_list = []
for col in uni_credit.columns[1:]:
    for row in uni_credit[col].values:
        uc_list.append(row)
        
# repeat 14 times per element
uc_rep_list = []
for element in uc_list:
    uc_rep_list.extend([element for i in range(14)])

crime1["Uni_credit"] = uc_rep_list

# replace nill or negligible value indicator ".." with 0
crime1['Uni_credit'] = crime1['Uni_credit'].replace("..", 0)
crime1['Uni_credit'] = [float(i) for i in crime1['Uni_credit']]

In [44]:
crime1.to_csv("crime1.csv", index = False)